In [13]:
import numpy as np
import pandas as pd
import string

import re
import nltk
import spacy
from spacy import displacy
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA, LEMMA
from spacy.tokens import Doc
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics
import pickle
nlp = spacy.load('en_core_web_lg')

In [14]:
def remove_stopwords(doc):
    indexes = []
    for index, token in enumerate(doc):
        if token.is_stop:
            indexes.append(index)
    np_array = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA, LEMMA])
    np_array = np.delete(np_array, indexes, axis = 0)
    doc2 = Doc(doc.vocab, words=[t.text for i, t in enumerate(doc) if i not in indexes])
    doc2.from_array([LOWER, POS, ENT_TYPE, IS_ALPHA, LEMMA], np_array)
    return doc2
def clean_header(text):
    text = re.sub(r'(From:\s+[^\n]+\n)', '', text)
    text = re.sub(r'(Subject:[^\n]+\n)', '', text)
    text = re.sub(r'(([\sA-Za-z0-9\-]+)?[A|a]rchive-name:[^\n]+\n)', '', text)
    text = re.sub(r'(Last-modified:[^\n]+\n)', '', text)
    text = re.sub(r'(Version:[^\n]+\n)', '', text)

    return text
def clean_text(text):        
    text = text.lower()
    text = text.strip()
    text = re.sub(re_url, '', text)
    text = re.sub(re_email, '', text)
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = re.sub(r'(\d+)', ' ', text)
    text = re.sub(r'(\s+)', ' ', text)
    
    return text

In [15]:
with open('103125','r',encoding='latin-1') as f:
    data=f.read()

In [16]:
data

"Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!ogicse!emory!gatech!usenet.ins.cwru.edu!cleveland.Freenet.Edu!ai598\nFrom: ai598@cleveland.Freenet.Edu (Mike Sturdevant)\nNewsgroups: rec.motorcycles\nSubject: Carrying crutches (was Re: Living\nMessage-ID: <1pqhkl$g48@usenet.INS.CWRU.Edu>\nDate: 6 Apr 93 00:07:49 GMT\nArticle-I.D.: usenet.1pqhkl$g48\nReferences: <1993Apr2.213402.10821@eos.arc.nasa.gov> <05APR93.02451203.0049@UNBVM1.CSD.UNB.CA> <1993Apr05.172836.36759@rchland.ibm.com>\nReply-To: ai598@cleveland.Freenet.Edu (Mike Sturdevant)\nOrganization: Case Western Reserve University, Cleveland, OH (USA)\nLines: 18\nNNTP-Posting-Host: slc10.ins.cwru.edu\n\n\nIn a previous article, pooder@rchland.vnet.ibm.com (Don Fearn) says:\n\n>\n>When I broke my right leg in two places (not a motorcycle accident -- a \n>_car_ accident; who woulda thunk it?) I put my crutches on Gretchen's rear\n>rack.  I'm getting the metal rod pulled out on Friday and going back on \n>the crutches for six wee

In [17]:
re_url = re.compile(r'(?:http|ftp|https)://(?:[\w_-]+(?:(?:\.[\w_-]+)+))(?:[\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?')
re_email = re.compile('(?:[a-z0-9!#$%&\'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&\'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])')                 

In [18]:
data = clean_text(clean_header(data))

In [19]:
data

'path cantaloupesrvcscmuedudasnewsharvardeduogicseemorygatechusenetinscwrueduclevelandfreeneteduai newsgroups recmotorcycles messageid date apr gmt articleid usenet pqhklg references replyto mike sturdevant organization case western reserve university cleveland oh usa lines nntppostinghost slc inscwruedu in a previous article don fearn says when i broke my right leg in two places not a motorcycle accident a car accident who woulda thunk it i put my crutches on gretchens rear rack im getting the metal rod pulled out on friday and going back on the crutches for six weeks i think crutches on the handlebars sound more aesthetically pleasing how did you attach them when i got my knee rebuilt i got back on the street bike asap i put the crutches on the rack and the passenger seat and they hung out back a long way just make sure theyre tied down tight in front and no problemo go fast take chances mike s'

In [20]:
data=remove_stopwords(nlp(data))

In [21]:
data

path cantaloupesrvcscmuedudasnewsharvardeduogicseemorygatechusenetinscwrueduclevelandfreeneteduai newsgroups recmotorcycles messageid date apr gmt articleid usenet pqhklg references replyto mike sturdevant organization case western reserve university cleveland oh usa lines nntppostinghost slc inscwruedu previous article don fearn says broke right leg places motorcycle accident car accident woulda thunk crutches gretchens rear rack m getting metal rod pulled friday going crutches weeks think crutches handlebars sound aesthetically pleasing attach got knee rebuilt got street bike asap crutches rack passenger seat hung long way sure tied tight problemo fast chances mike s 

In [22]:
data= ' '.join(token.lemma_ for token in data)

In [23]:
data

'path cantaloupesrvcscmuedudasnewsharvardeduogicseemorygatechusenetinscwrueduclevelandfreeneteduai newsgroup recmotorcycle messageid date apr gmt articleid usenet pqhklg reference replyto mike sturdevant organization case western reserve university cleveland oh usa line nntppostinghost slc inscwruedu previous article don fearn say break right leg place motorcycle accident car accident woulda thunk crutch gretchens rear rack m get metal rod pull friday go crutch week think crutch handlebar sound aesthetically pleasing attach get knee rebuild get street bike asap crutch rack passenger seat hang long way sure tie tight problemo fast chance mike s'

In [24]:
names = ['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc',
       'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware',
       'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles',
       'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt',
       'sci.electronics', 'sci.med', 'sci.space',
       'soc.religion.christian', 'talk.politics.guns',
       'talk.politics.mideast', 'talk.politics.misc',
       'talk.religion.misc']

In [36]:
vectorizer = pickle.load(open('TfidfVectorizer.pkl', 'rb'))

C:\Users\mhmed\AppData\Local\Temp\ipykernel_7312\388543793.py:1: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  vectorizer = pickle.load(open('TfidfVectorizer.pkl', 'rb'))


In [ ]:
data=vectorizer.transform(data)

In [33]:
bestmodel = pickle.load(open('XGBClassifierModel.pkl', 'rb'))